In [ ]:
import arcpy
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
import os
import json
from datetime import datetime
from uuid import UUID
from pprint import pprint

### Connect

In [ ]:
username = "analyst"
password = "Esri.4.GIS"
url = "https://ebase.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
gis = GIS(r"https://ebase.ad.local/portal", username, password)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

### Query

In [1]:
def createRelationships(queryResults):
    for p, d in queryResults:
        print("Related nodes:")
        print(p['_properties']['PlantName'])
        print(p['_id'])
        print(p['_properties']['Material'])
        print(d['_properties']['DistributorName'])
        print(d['_id'])
        print(d['_properties']['Material'])
        print()
        add_relationship = {
            "_objectType": "relationship",
            "_typeName": "SellsTo",
            "_originEntityId": UUID(f"{d['_id']}"), # Distributor ID
            "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
        }
        print(add_relationship)
        knowledge_graph.apply_edits(adds=[add_relationship])
        print("EDITS MADE")
    print("All done!")

In [ ]:
def submitQuery(distributorName):
    opQuery = """MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "{0}")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "Distributor 2"
    RETURN p, d""".format(distributorName)
    print(opQuery)

    queryResults = knowledge_graph.query(opQuery)
    return queryResults

In [ ]:
queryNodes =submitQuery("Distributor 2")
createRelationships(queryNodes)